# 🎯 Problema de Negócio:

* A Vale S.A. (VALE3) é uma das principais empresas de mineração listadas na B3, com forte impacto no portfólio de investidores institucionais e pessoa física no Brasil. Para um gestor de investimentos ou analista de mercado, antecipar a trajetória do preço de fechamento (Close) e entender os padrões de volume de negociação é crucial para:

    1. Tomar decisões de compra/venda com melhor relação risco-retorno;

    2. Ajustar a alocação de ativos de curto a médio prazo;

    3. Desenvolver estratégias de trading baseadas em sinais estatísticos e de machine learning;

    4. Gerenciar o risco por meio de previsões confiáveis e métricas de incerteza.

* Objetivo Geral: Desenvolver um modelo preditivo de séries temporais para o preço de fechamento diário de VALE3, de modo a gerar forecasts de 7 dias futuros e embasar decisões de investimento.

* Importação da base de dados:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import RFECV
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)
import warnings
warnings.filterwarnings("ignore")

# 1. Coleta e tratamento de dados.

In [2]:
# Coletando e tratando os dados das ações VALE3:
vale = yf.download('VALE3.SA', start='2022-01-03', end='2025-07-23', multi_level_index=False)
vale = vale[['Close', 'Open', 'Volume']]
vale.reset_index(inplace=True)
vale['Date'] = pd.to_datetime(vale['Date'])
vale.set_index('Date', inplace=True)
vale.rename(columns={
    'Close': 'Close_Preco_VALE3',
    'Open': 'Open_Preco_VALE3',
    'Volume': 'Variacao_Preco_VALE3'
}, inplace=True)

# Vamos pegar os dados do preço do minério de ferro:
minerio_ferro = pd.read_csv('Dados Históricos - Minério de ferro refinado 62% Fe CFR Futuros.csv')
minerio_ferro.reset_index(inplace=True)
minerio_ferro['Data'] = pd.to_datetime(minerio_ferro['Data'], dayfirst=True)
minerio_ferro.set_index('Data', inplace=True)
minerio_ferro = minerio_ferro[['Último','Abertura', 'Var%']]
# Invertendo a ordem das datas:
minerio_ferro = minerio_ferro.iloc[::1]
minerio_ferro.rename(columns={
    'Último': 'Close_Preco_Ferro',
    'Abertura': 'Open_Preco_Ferro',
    'Var%': 'Variacao_ferro'
}, inplace=True)

# Unindo o dataframe da VALE3 com o dataset de Minerrio de Ferro:
df = pd.merge(vale, minerio_ferro, left_index=True, right_index=True, suffixes=('_VALE3', '_Minerio'))

# Variação percentual do preço de fechamento da VALE3:
df['Variação_VALE3'] = df['Close_Preco_VALE3'].pct_change() * 100
# Trocando nan por 0%:
df['Variação_VALE3'].fillna(0, inplace=True)

# Vamos transformar as colunas Close_Minerio, Open_Minerio e Variação_Minerio em float:
df['Close_Preco_Ferro'] = df['Close_Preco_Ferro'].str.replace(',', '.', regex=False).astype(float)
df['Open_Preco_Ferro'] = df['Open_Preco_Ferro'].str.replace(',', '.', regex=False).astype(float)
df['Variacao_ferro'] = df['Variacao_ferro'].str.replace('%', '', regex=False).str.replace(',', '.', regex=False).astype(float)

# Variação percentual do preço de fechamento do minério de ferro:
df['Variação_Preco_Fechamento_Ferro'] = df['Close_Preco_Ferro'].pct_change() * 100 
# Trocando nan por 7,02%:
df['Variação_Preco_Fechamento_Ferro'].fillna(7.02, inplace=True)

df

[*********************100%***********************]  1 of 1 completed


,Close_Preco_VALE3,Open_Preco_VALE3,Variacao_Preco_VALE3,Close_Preco_Ferro,Open_Preco_Ferro,Variacao_ferro,Variação_VALE3,Variação_Preco_Fechamento_Ferro
2022-01-03,57.766411,58.507006,18557200,120.40,120.40,7.02,0.000000,7.020000
2022-01-04,57.085072,58.144123,18178700,120.91,120.91,0.42,-1.179473,0.423588
2022-01-05,57.625702,57.299844,22039000,124.14,124.14,2.67,0.947061,2.671408
2022-01-06,58.788433,58.240394,22044100,125.94,125.94,1.45,2.017730,1.449976
2022-01-07,62.209984,59.543843,35213100,126.21,126.21,0.21,5.820109,0.214388
...,...,...,...,...,...,...,...,...
2025-07-16,54.400002,54.099998,26394100,97.06,97.06,0.58,0.908925,0.580311
2025-07-17,54.299999,54.400002,20252800,97.18,97.18,0.12,-0.183828,0.123635
2025-07-18,54.560001,54.049999,23714000,97.22,97.22,0.04,0.478825,0.041161
2025-07-21,56.049999,55.450001,40484000,97.84,97.84,0.64,2.730934,0.637729


In [3]:
df.reset_index(inplace=True)
df['index'] = pd.to_datetime(df['index'], dayfirst=True)
df.rename(columns={'index': 'Data'}, inplace=True)
df.set_index('Data', inplace=True)
df

,Close_Preco_VALE3,Open_Preco_VALE3,Variacao_Preco_VALE3,Close_Preco_Ferro,Open_Preco_Ferro,Variacao_ferro,Variação_VALE3,Variação_Preco_Fechamento_Ferro
Data,,,,,,,,
2022-01-03,57.766411,58.507006,18557200,120.40,120.40,7.02,0.000000,7.020000
2022-01-04,57.085072,58.144123,18178700,120.91,120.91,0.42,-1.179473,0.423588
2022-01-05,57.625702,57.299844,22039000,124.14,124.14,2.67,0.947061,2.671408
2022-01-06,58.788433,58.240394,22044100,125.94,125.94,1.45,2.017730,1.449976
2022-01-07,62.209984,59.543843,35213100,126.21,126.21,0.21,5.820109,0.214388
...,...,...,...,...,...,...,...,...
2025-07-16,54.400002,54.099998,26394100,97.06,97.06,0.58,0.908925,0.580311
2025-07-17,54.299999,54.400002,20252800,97.18,97.18,0.12,-0.183828,0.123635
2025-07-18,54.560001,54.049999,23714000,97.22,97.22,0.04,0.478825,0.041161


# 2. Adicionando features ao nosso dataframe.

* 1. Vamos começar adicionando features de preço:

* Vamos adicionar features temporais de 7 dias, 20 dias e 200 dias para para suavizar as oscilações de preços, ajudando a identificar padrões mais claros na evolução do valor de ativos:

In [4]:
# Médias móveis para os preços da VALE3:
df['MM_7D_VALE3'] = df['Close_Preco_VALE3'].rolling(window=7, min_periods=1).mean()
df['MM_20D_VALE3'] = df['Close_Preco_VALE3'].rolling(window=20, min_periods=1).mean()
df['MM_200D_VALE3'] = df['Close_Preco_VALE3'].rolling(window=200, min_periods=1).mean()

# Médias móveis para os preços da commodite do minério de ferro:
df['MM_7D_Ferro'] = df['Close_Preco_Ferro'].rolling(window=7, min_periods=1).mean()
df['MM_20D_Ferro'] = df['Close_Preco_Ferro'].rolling(window=20, min_periods=1).mean()
df['MM_200D_Ferro'] = df['Close_Preco_Ferro'].rolling(window=200, min_periods=1).mean()

df[['MM_7D_VALE3','MM_20D_VALE3','MM_200D_VALE3','MM_7D_Ferro','MM_20D_Ferro','MM_200D_Ferro']]

,MM_7D_VALE3,MM_20D_VALE3,MM_200D_VALE3,MM_7D_Ferro,MM_20D_Ferro,MM_200D_Ferro
Data,,,,,,
2022-01-03,57.766411,57.766411,57.766411,120.400000,120.400000,120.400000
2022-01-04,57.425741,57.425741,57.425741,120.655000,120.655000,120.655000
2022-01-05,57.492395,57.492395,57.492395,121.816667,121.816667,121.816667
2022-01-06,57.816404,57.816404,57.816404,122.847500,122.847500,122.847500
2022-01-07,58.695120,58.695120,58.695120,123.520000,123.520000,123.520000
...,...,...,...,...,...,...
2025-07-16,54.795715,53.166000,54.648418,96.380000,95.317000,101.126700
2025-07-17,54.755714,53.310500,54.640896,96.612857,95.433000,101.152900
2025-07-18,54.830000,53.476500,54.636009,96.884286,95.558500,101.179100


* Vamos adicionar o retorno diário e os retornos de 7 dias anterior:

In [5]:
df['Retorno_1D'] = df['Close_Preco_VALE3'].pct_change() * 100
df['Retorno_1D'].fillna(0, inplace=True)
df['Retorno_7D'] = df['Close_Preco_VALE3'].pct_change(7) * 100 
df['Retorno_7D'].fillna(0, inplace=True)
df['Retorno_21D'] = df['Close_Preco_Ferro'].pct_change(21) * 100
df['Retorno_21D'].fillna(0, inplace=True)

* Vamos adicionar a volatilidade das ações da VALE3 para que o XGBoost capture os padrões das volatilidades de 7 dias e 21 dias:

In [6]:
df['Volatilidade_7D_VALE3'] = np.std(df['Retorno_7D'])
df['Volatilidadde_21D_VALE3'] = np.std(df['Retorno_21D'])
df['Volatilidade_7D_VALE3'].fillna(0, inplace=True)
df['Volatilidadde_21D_VALE3'].fillna(0, inplace=True)

* 2. Agora vamos adicionar features de indicadores tecnicos, que são excelemtes para Tradings:

* Começando pelo Relative Strength Index(14 dias). Um indicador usado para avaliar a força de um movimento
de preço e determinar se um ativo está sobrecarregado ou sobrevendido:

In [7]:
def calcular_rsi(series, window=14):
    delta = series.diff()

    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_again = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()

    rs = avg_again / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

df['RSI_14'] = calcular_rsi(df['Close_Preco_VALE3'], window=14)
df['RSI_14'].fillna(df['RSI_14'].mean(), inplace=True)

* O Moving Average Convergence Divergence ajuda a identificar mudanças na força, direção, momento e duração de uma tendência de ações:

In [ ]:
MME_12 = df['Close_Preco_VALE3'].ewm(span=12, adjust=False).mean()
MME_26 = df['Close_Preco_VALE3'].ewm(span=26, adjust=False).mean()

df['MACD'] = MME_12 - MME_26

df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

* As bandas de Bollinger são indicadores usados para medir a volatilidade do mercado e identificar potenciais pontos de compra e venda de ativos:

In [9]:
# primeiro pegamos as medias móveis de 20 dias:
medias_20D = df['MM_20D_VALE3']
# E o desvio padrão de 20 dias também:
desvio_20D = df['Close_Preco_VALE3'].rolling(window=20, min_periods=1).std()

# Assim podemos calcular as bandas:
df['banda_media'] = medias_20D
df['banda_superior'] = medias_20D + (2 * desvio_20D)
df['banda_inferior'] = medias_20D - (2 * desvio_20D)

# Vamos completar os valores NaN com a média dos dados para cada coluna:
df['banda_inferior'].fillna(df['banda_inferior'].mean(), inplace=True)
df['banda_superior'].fillna(df['banda_superior'].mean(), inplace=True)
df[['banda_media', 'banda_superior', 'banda_inferior']]

,banda_media,banda_superior,banda_inferior
Data,,,
2022-01-03,57.766411,63.308963,55.820816
2022-01-04,57.425741,58.389300,56.462182
2022-01-05,57.492395,58.211794,56.772995
2022-01-06,57.816404,59.239338,56.393471
2022-01-07,58.695120,62.813541,54.576700
...,...,...,...
2025-07-16,53.166000,57.052611,49.279389
2025-07-17,53.310500,57.136646,49.484354
2025-07-18,53.476500,57.211407,49.741593


* O Rolling Sharpe Ratio, ou Razão de Sharpe Móvel, é usado para avaliar o desempenho de um investimento ao longo do tempo, ajustado ao risco:  

In [10]:
def Rolling_Sharpe_Ratio(returns, window=21, risk_free_rate=0.0, trading_days=252):
    # Converter taxa livre de risco para diária:
    rf_daily = (1 + risk_free_rate)**(1/trading_days) - 1

    # Calcular retornos excedentes:
    excess_return = returns - rf_daily

    # Média móvel dos retornos excedentes:
    mean_returns = excess_return.rolling(window).mean()

    # Desvio padrão móvel dos retornos excedentes:
    std_returns = excess_return.rolling(window).std()

    # Sharpe Ratio anualizado:
    sharpe_ratio = mean_returns / std_returns * np.sqrt(trading_days)

    return sharpe_ratio

# Calculando o sharpe ratio de 21 dias( 21 dias = 1 mês de trading):
df['Sharpe_21D'] = Rolling_Sharpe_Ratio(
    returns=df['Retorno_1D'],
    window=21,
    risk_free_rate= 0.15, # 15% ao ano (SELIC atual)
    trading_days=252
)